In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import FloatSlider

from PIL import Image, ImageFilter
import PIL
from IPython import display

from matplotlib.pyplot import imshow
%matplotlib inline

import pickle

In [2]:
from gen_pic_utils import *
from gen_pic_utils import *
from gen_utils import *
from image_generation_utils import *
from update_library import *

library = update_library()

## Select road type

In [7]:
road_type = 74
car_type = 28

image_size = (1242,375)

In [8]:
def save(filename,data):
    with open(filename, 'w+') as f:
        pickle.dump(data, f)
        
def load(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

In [9]:
# Init variables
poly_verts = 4
polygon = [(image_size[0],image_size[1]),(image_size[0],1),(1,1),(1,image_size[1])]
scales = [1, 1]
    
# Slider parameters
point_widget = FloatSlider(min=0, max=poly_verts-1, step=1, value=0)
x_widget = FloatSlider(min=1, max=image_size[0], step=1, value=image_size[0])
y_widget = FloatSlider(min=1, max=image_size[1], step=1, value=image_size[1])
car_widget = FloatSlider(min=0, max=1, step=1, value=0)
scale_widget = FloatSlider(min=0.1, max=2.0, step=0.1, value=1)
car_x_pos_widget = FloatSlider(min=0.0, max=1.0, step=0.1, value=0.5)

def update_car_conf_widgets(*args):
    (x,y) = polygon[int(point_widget.value)]
    x_widget.value = x
    y_widget.value = y
    
def update_car_scale_widgets(*args):    
    scale = scales[int(car_widget.value)]
    scale_widget.value = scale
    
car_widget.observe(update_car_scale_widgets, 'value')    
point_widget.observe(update_car_conf_widgets, 'value')

def update_point_widgets(point,x,y,car_pos,scale,car_x_pos):
    
    point_id = int(point_widget.value)
    polygon[point_id] = (x_widget.value,y_widget.value)
    tmp_pt = list(polygon[3 - point_id])
    tmp_pt[1] = y_widget.value
    polygon[3 - point_id] = tuple(tmp_pt)    
        
    polygon[int(point_widget.value)] = (x_widget.value,y_widget.value)
    scales[int(car_pos)] = scale       
    
    library = update_library()

    background = library.background_objects[road_type]
    background_copy = background.image.copy()

    # remove alpha channel from background (if present)
    if background_copy.mode in ('RGBA', 'LA') or \
            (background_copy.mode == 'P' and
                     'transparency' in background_copy.info):
        background_no_alpha = \
            Image.new("RGB", background_copy.size, (255, 255, 255))
        background_no_alpha.paste(background_copy,
                                  mask=background_copy.split()[3])
                                # 3 is the alpha channel
    else:
        background_no_alpha = background_copy
        
    road = background_no_alpha 
    car = library.foreground_objects[car_type].image
    
    lrx = polygon[0][0]
    lry = polygon[0][1]
    llx = polygon[3][0]
    
    urx = polygon[1][0]
    ury = polygon[1][1]
    ulx = polygon[2][0]
    
    car = car.resize((int(car.size[0]*scale), int(car.size[1]*scale)), PIL.Image.ANTIALIAS)     
    
    if(car_pos == 0):
        car_x = int(llx + (lrx-llx)*car_x_pos - car.size[0]/2)
        car_y = int(lry - (car.size[1]/2))
    else:
        car_x = int(ulx + (urx-ulx)*car_x_pos - car.size[0]/2)
        car_y = int(ury - (car.size[1]/2))
    
    road.paste(car, (car_x,car_y), car)    
    draw = ImageDraw.Draw(road)
    
    eps = 5
    draw.ellipse((x-eps, y-eps, x+eps, y+eps), fill = 'blue', outline ='blue')
    draw.polygon(polygon, fill=None, outline=(255,0,0,255))      
        
    display.display(road)
    

In [10]:
interact(update_point_widgets,point=point_widget, x=x_widget, y=y_widget, car_pos=car_widget, scale=scale_widget, car_x_pos=car_x_pos_widget);

interactive(children=(FloatSlider(value=0.0, description=u'point', max=3.0, step=1.0), FloatSlider(value=1242.0, description=u'x', max=1242.0, min=1.0, step=1.0), FloatSlider(value=375.0, description=u'y', max=375.0, min=1.0, step=1.0), FloatSlider(value=0.0, description=u'car_pos', max=1.0, step=1.0), FloatSlider(value=1.0, description=u'scale', max=2.0, min=0.1), FloatSlider(value=0.5, description=u'car_x_pos', max=1.0), Output()), _dom_classes=('widget-interact',))

In [11]:
(polygon,scales)

([(677.0, 285.0), (651.0, 157.0), (572.0, 157.0), (554.0, 285.0)], [1.1, 0.2])

In [12]:
configs = load('scene_configs_py2')
configs[road_type] = (polygon,scales)
save('scene_configs_py2',configs)

In [13]:
configs

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ([(786.0, 286.0), (651.0, 226.0), (602.0, 226.0), (495.0, 286.0)],
  [1.0, 0.2]),
 ([(809.0, 261.0), (689.0, 182.0), (558.0, 182.0), (450.0, 261.0)],
  [1.4, 0.3]),
 ([(634.0, 285.0), (637.0, 163.0), (619.0, 163.0), (448.0, 285.0)],
  [1.0, 0.2]),
 ([(883.0, 257.0), (805.0, 171.0), (631.0, 171.0), (566.0, 257.0)],
  [1.3, 0.6]),
 ([(887.0, 267.0), (734.0, 112.0), (651.0, 112.0), (476.0, 267.0)],
  [1.3, 0.3]),
 ([(802.0, 281.0), (678.0, 171.0), (564.0, 171.0), (459.0, 281.0)],
  [1.4, 0.5]),
 ([(825.0, 290.0), (680.0, 197.0), (573.0, 197.0), (379.0, 290.0)],
  [1.1, 0.3]),
 ([(745.0, 281.0), (640.0, 200.0), (592.0, 200.0), (479.0, 281.0)],
  [1.3, 0.3]),
 ([(719.0, 287.0), (633.0, 173.0), (602.0, 173.0), (537.0, 287.0)],
  [1.3, 

In [ ]:
[(782.0, 306.0), (634.0, 214.0), (595.0, 214.0), (345.0, 306.0)], [1.0, 0.2])